In [56]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import math

# load the train data
dataframe = read_csv('dataset/train.csv',usecols=[i for i in range(0,211)], engine='python', header=0)
trainDataOriginal = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

In [57]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [58]:
#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]
# normalize the dataset
scalerInput = MinMaxScaler(feature_range=(0, 1))
scalerOutput = MinMaxScaler(feature_range=(0, 1))

trainX = scalerInput.fit_transform(trainX)
trainY = scalerOutput.fit_transform(trainY)
testX = scalerInput.fit_transform(testX)

#Put it into batches
trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1]))

print trainX.shape
print trainY.shape
print testX.shape

(625, 64, 146)
(625, 64, 64)
(1875, 64, 146)


In [70]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense

# create the model
Opt = adam(0.01, 0.01, 0.001, 1e-08, 0.0)
model = Sequential()
model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
print model.output_shape
model.add(Dense(output_dim, activation='linear'))
print model.output_shape
#
#Configures the model for training.
model.compile(loss="mean_squared_error",optimizer=Opt)
#Trains the model for a fixed number of epochs (iterations on a dataset).
model.fit(trainX,trainY,batch_size=batch_size,epochs=100)
#Make predictions
testY = model.predict(testX)

(None, 64, 64)
(None, 64, 64)
Epoch 1/100
625/625 [==============================] - 3s - loss: 0.0969     
Epoch 2/100
625/625 [==============================] - 1s - loss: 0.0334     
Epoch 3/100
625/625 [==============================] - 1s - loss: 0.0299     
Epoch 4/100
625/625 [==============================] - 1s - loss: 0.0260     - ETA: 0s - loss: 0.0
Epoch 5/100
625/625 [==============================] - 2s - loss: 0.0236     
Epoch 6/100
625/625 [==============================] - 2s - loss: 0.0217     
Epoch 7/100
625/625 [==============================] - 1s - loss: 0.0202     
Epoch 8/100
625/625 [==============================] - 1s - loss: 0.0189     
Epoch 9/100
625/625 [==============================] - 1s - loss: 0.0183     
Epoch 10/100
625/625 [==============================] - 1s - loss: 0.0176     
Epoch 11/100
625/625 [==============================] - 1s - loss: 0.0171     
Epoch 12/100
625/625 [==============================] - 1s - loss: 0.0170     
Epoch 13/1

In [71]:
testY = testY.reshape(len(testData),output_dim)
testY = scalerOutput.inverse_transform(testY)
print testY


[[ 0.23487312  0.32881862  0.14228311 ...,  0.39555866  0.20155902
   0.34462774]
 [ 0.23566741  0.3496657   0.14251682 ...,  0.42910147  0.26657069
   0.38739929]
 [ 0.2400226   0.36789131  0.14808173 ...,  0.44355452  0.29409221
   0.40885851]
 ..., 
 [ 0.23651713  0.38241738  0.15046889 ...,  0.45259583  0.22327037
   0.34033272]
 [ 0.23614264  0.37547845  0.15725744 ...,  0.45287639  0.28235263
   0.3978025 ]
 [ 0.23016575  0.37838316  0.15058219 ...,  0.44899976  0.26578835
   0.37889063]]
